### 1. import

In [52]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *

### 2. Create a Quantum Class with Qiskit

In [53]:
class QuantumCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit 
    """
    
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
        # ---------------------------

        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts(self._circuit)
        
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        
        # Compute probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        
        return np.array([expectation])

### 3. Create a "Quantum-Classical Class" with PyTorch

In [54]:
class HybridFunction(Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(1, backend, shots)
        self.shift = shift
        
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

### 4. Data Loading and Preprocessing

In [78]:
import pandas as pd
df = pd.read_csv("./data2.csv")
df.head()

,id,name,date,manner_of_death,armed,age,gender,city,state,signs_of_mental_illness,...,share_hispanic,Race_Asian,Race_Black,Race_Hispanic,Race_Native_American,Race_Other,Race_White,threat_attack,threat_other,threat_undem
0,3,Tim Elliot,1/15/02,0,1,53.0,1,Shelton,WA,1,...,19.2,1,0,0,0,0,0,1,0,0
1,4,Lewis Lee Lembke,1/15/02,0,1,47.0,1,Aloha,OR,0,...,21.1,0,0,0,0,0,1,1,0,0
2,890,Phyllis Ilene Jepsen,10/15/02,0,1,55.0,0,Aloha,OR,1,...,21.1,0,0,0,0,0,1,0,1,0
3,5,John Paul Quintero,1/15/03,1,0,23.0,1,Wichita,KS,0,...,15.3,0,0,1,0,0,0,0,1,0
4,765,Nicholas Garner,8/15/22,0,1,26.0,1,Wichita,KS,0,...,15.3,0,0,0,0,0,1,1,0,0


In [87]:
health_df = df[df['signs_of_mental_illness']==0]
unhealth_df = df[df['signs_of_mental_illness']==1]

In [93]:
train_200sample = pd.concat([health_df.sample(n=100, random_state=22),unhealth_df.sample(n=100, random_state=22)])
test_200sample = pd.concat([health_df.sample(n=100, random_state=20),unhealth_df.sample(n=100, random_state=20)])

# shuffle
train_200sample = train_200sample.sample(frac=1)
test_200sample = test_200sample.sample(frac=1)

#the list of predictor variables to be included
var_list = ['flee','manner_of_death','gender','Race_Asian',
            'Race_Black','Race_Hispanic','Race_Native_American','Race_White']

train_X = train_200sample[var_list]
train_y = train_200sample['signs_of_mental_illness']

test_X = test_200sample[var_list]
test_y = test_200sample['signs_of_mental_illness']

In [99]:
train_y.iloc[0]

1

In [56]:
#var_list = ['flee', 'age', 'manner_of_death', 'gender', 'armed', 'Race_Asian', 'Race_Black', 'Race_Hispanic', 'Race_Native_American', 'Race_Other', 'Race_White'] 
#X = data[var_list]
#y = data['signs_of_mental_illness']
#train_loader = torch.utils.data.DataLoader(X, batch_size=1, shuffle=True)

### 5. Creating the Hybrid Neural Network

In [110]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(8, 10)
        self.fc2 = nn.Linear(10, 7)
        self.fc3 = nn.Linear(7, 5)
        self.fc4 = nn.Linear(5, 1)
        self.hybrid = Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.hybrid(x)
        return torch.cat((x, 1 - x), -1)

### 6. Train the neural network

In [111]:
from torch.autograd import Variable

model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.NLLLoss()

epochs = 10
loss_list = []

model.train()
for epoch in range(epochs):
    total_loss = []
    for i in range(len(train_X)):
        item = train_X.iloc[i,:]
        item = item.to_numpy()
        item = np.reshape(item, (-1, len(item)))
        item = Variable(torch.from_numpy(item).float())
        #item = np.array[item]
        #item = item.double()
        
        label = torch.tensor(np.array([train_y.iloc[i]]))
        #label = torch.FloatTensor(one_hot_encoding(label)).view(1, -1)
        label = label.float()
        label = label.type(torch.LongTensor)
        #print(label)
        
        optimizer.zero_grad()
        # Forward pass
        output = model(item)
        # Calculating loss
        loss = loss_func(output, label)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1]))

Traceback (most recent call last):
  File "<ipython-input-111-335c74a25519>", line 29, in <module>
    output = model(item)
  File "/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "<ipython-input-110-2bb784bd366a>", line 16, in forward
    x = F.Sigmoid(self.fc4(x))
AttributeError: module 'torch.nn.functional' has no attribute 'Sigmoid'

Use %tb to get the full traceback.


In [ ]:
plt.plot(loss_list)
plt.title('Hybrid NN Training Convergence')
plt.xlabel('Training Iterations')
plt.ylabel('Neg Log Likelihood Loss')

### 7. Test the Neural Network

In [109]:
model.eval()
with torch.no_grad():
    
    correct = 0
    for i in range(len(test_X)):
        item = test_X.iloc[i,:]
        item = item.to_numpy()
        item = np.reshape(item, (-1, len(item)))
        item = Variable(torch.from_numpy(item).float())
        output = model(item)
        
        label = torch.tensor(np.array([test_y.iloc[i]]))
        label = label.float()
        label = label.type(torch.LongTensor)
        
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(label.view_as(pred)).sum().item()
        
        loss = loss_func(output, label)
        total_loss.append(loss.item())
        
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_X) * 100)
        )

Performance on test data:
	Loss: -0.5298
	Accuracy: 50.0%


In [75]:
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(X) * 100)
        )

Performance on test data:
	Loss: -0.5014
	Accuracy: 54.0%
